In [3]:
import torch
import librosa
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

In [4]:
LANG_ID = "ru"
MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-russian"
SAMPLES = 5

In [11]:
test_dataset = load_dataset("Nexdata/Russian_Speech_Data_by_Mobile_Phone")

README.md:   0%|          | 0.00/3.23k [00:00<?, ?B/s]

C:\Users\vadim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vadim\.cache\huggingface\hub\datasets--Nexdata--Russian_Speech_Data_by_Mobile_Phone. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


G01025S1001.wav:   0%|          | 0.00/196k [00:00<?, ?B/s]

G01751S2178.wav:   0%|          | 0.00/278k [00:00<?, ?B/s]

G02059S3280.wav:   0%|          | 0.00/157k [00:00<?, ?B/s]

G10343S4403.wav:   0%|          | 0.00/185k [00:00<?, ?B/s]

G10343S5430.wav:   0%|          | 0.00/145k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5 [00:00<?, ? examples/s]

In [17]:
test_dataset['train'][0]

{'audio': {'path': 'C:\\Users\\vadim\\.cache\\huggingface\\hub\\datasets--Nexdata--Russian_Speech_Data_by_Mobile_Phone\\snapshots\\84d24169b3886131373fb08ea96fce7399dd3514\\G01025S1001.wav',
  'array': array([ 3.05175781e-04, -3.66210938e-04,  3.05175781e-05, ...,
          9.15527344e-05,  1.22070312e-04,  0.00000000e+00]),
  'sampling_rate': 16000}}

In [13]:
processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

preprocessor_config.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

C:\Users\vadim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vadim\.cache\huggingface\hub\models--jonatasgrosman--wav2vec2-large-xlsr-53-russian. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

In [23]:
# Preprocessing the datasets.
# We need to read the audio files as arrays
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = librosa.load(batch['audio']["path"], sr=16_000)
    batch["speech"] = speech_array
    return batch

In [24]:
test_dataset = test_dataset.map(speech_file_to_array_fn)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [31]:
test_dataset['train'][0]

{'audio': {'path': None,
  'array': array([ 3.05175781e-04, -3.66210938e-04,  3.05175781e-05, ...,
          9.15527344e-05,  1.22070312e-04,  0.00000000e+00]),
  'sampling_rate': 16000},
 'speech': [0.00030517578125,
  -0.0003662109375,
  3.0517578125e-05,
  0.0006103515625,
  -0.000518798828125,
  0.000396728515625,
  6.103515625e-05,
  3.0517578125e-05,
  0.0001220703125,
  0.0001220703125,
  0.000213623046875,
  -0.000335693359375,
  6.103515625e-05,
  -3.0517578125e-05,
  -0.000457763671875,
  -0.000152587890625,
  -9.1552734375e-05,
  -0.000518798828125,
  0.000152587890625,
  -0.000213623046875,
  -3.0517578125e-05,
  0.00018310546875,
  -0.000152587890625,
  -0.000152587890625,
  0.00018310546875,
  6.103515625e-05,
  -3.0517578125e-05,
  6.103515625e-05,
  -3.0517578125e-05,
  6.103515625e-05,
  9.1552734375e-05,
  -3.0517578125e-05,
  0.0003662109375,
  0.000213623046875,
  -0.0001220703125,
  0.000244140625,
  0.000152587890625,
  -0.000152587890625,
  3.0517578125e-05,
  0.

In [33]:
inputs = processor(test_dataset['train']["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

In [34]:
inputs

{'input_values': tensor([[ 0.0090, -0.0108,  0.0009,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0194, -0.0139, -0.0121,  ..., -0.0342, -0.0434, -0.0305],
        [-0.0047, -0.0030,  0.0107,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0041, -0.0088, -0.0018,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0213,  0.0109,  0.0150,  ...,  0.0000,  0.0000,  0.0000]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], dtype=torch.int32)}

In [35]:
with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

In [36]:
predicted_ids = torch.argmax(logits, dim=-1)
predicted_sentences = processor.batch_decode(predicted_ids)

In [38]:
for i, predicted_sentence in enumerate(predicted_sentences):
    print("-" * 100)
    print("Prediction:", predicted_sentence)

----------------------------------------------------------------------------------------------------
Prediction: еще одно такое замечание я скажу в анхе переключить на два по-настоящему
----------------------------------------------------------------------------------------------------
Prediction: там отметили что дети получили медицинскую помощь в случае необходимости им окажут также психологическую поддержку
----------------------------------------------------------------------------------------------------
Prediction: пожалуйста дай сведения а наличии товары свитеры у продавца п
----------------------------------------------------------------------------------------------------
Prediction: пожалуйста установи таймир обратного осчета для мультиварки на полчаса
----------------------------------------------------------------------------------------------------
Prediction: сколько времени добираться от моего дома до ближайшего банка
